In [7]:
import pandas as pd
import numpy as np
from predict_utils  import final_cont_features, final_cat_features, final_features, incident_features, capital_features

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor, Pool

from datetime import timedelta

from tqdm import tqdm

import pickle

predict_features = incident_features + capital_features
length_features = [i.replace('between', 'length') for i in predict_features]

In [10]:
data = pd.read_csv('data/to_train.csv', index_col='unom')
data

/tmp/ipykernel_255109/3478314917.py:1: DtypeWarning: Columns (153,162,165,168) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/to_train.csv', index_col='unom')


,year,floors,padiks,apts,S,S_l,S_u,wear,walls,break,...,ВДСК_c_between,Дорога_length,Дорога_date,Дорога_between,КАН_c_length,КАН_c_date,КАН_c_between,ГАЗ_c_length,ГАЗ_c_date,ГАЗ_c_between
unom,,,,,,,,,,,,,,,,,,,,,
64148,2005,18,2.0,148.0,8266.0,8220.0,46.0,NaN,монолитные (ж-б),нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19485,1955,4,2.0,20.0,1240.0,1070.0,170.0,NaN,кирпичные,нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18533,1960,5,3.0,60.0,2518.0,2492.0,26.0,NaN,кирпичные,нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22830,1973,9,12.0,431.0,21103.0,21088.0,15.0,0.0,панельные,нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12331,1968,5,4.0,64.0,4133.0,2697.0,1436.0,NaN,кирпичные,нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5012070,2017,20,NaN,NaN,42316.0,24468.0,17848.0,NaN,бетонные,нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5012968,2017,18,NaN,NaN,20306.0,19068.0,1238.0,NaN,монолитные (ж-б),нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5024600,2018,20,NaN,NaN,13820.0,12375.0,1445.0,NaN,монолитные (ж-б),нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Impute and Scale

In [11]:
# Impute continuous features using KNN
imputer_continuous = KNNImputer(n_neighbors=5)  # You can adjust the value of n_neighbors as per your preference
scaler_continuous = StandardScaler()
data[final_cont_features] = imputer_continuous.fit_transform(data[final_cont_features])
data[final_cont_features] = scaler_continuous.fit_transform(data[final_cont_features])

imputer_categorical = dict()
imputer_length = dict()
# Impute categorical features using mode imputation
for feature in final_cat_features:
    mode_value = data[feature].mode().values[0]
    imputer_categorical[feature] =  mode_value
    data[feature].fillna(mode_value, inplace=True)
    data[feature] = data[feature].replace(np.nan, mode_value)
    
for feature in length_features:
    mode_value = data[feature].mean()
    imputer_length[feature] =  mode_value
    
data[final_cat_features] = data[final_cat_features].astype(str) 

In [12]:
with open(f'models/imputer_continuous.pickle', 'wb') as f:
    pickle.dump(imputer_continuous, f)
with open(f'models/scaler_continuous.pickle', 'wb') as f:
    pickle.dump(scaler_continuous, f)
with open(f'models/imputer_categorical.pickle', 'wb') as f:
    pickle.dump(imputer_categorical, f)
with open(f'models/imputer_length.pickle', 'wb') as f:
    pickle.dump(imputer_length, f)

## Predictions

In [5]:
for cur_feat in tqdm(incident_features + capital_features):

    cur_columns = final_features + [cur_feat]
    cur_data = data.copy(deep=True)[cur_columns]
    cur_data = cur_data[~cur_data[cur_feat].isna()]
    cur_data.head()

    y = cur_data[[cur_feat]]
    X = cur_data.drop([cur_feat], axis=1)

    data_pool = Pool(X, y, cat_features=final_cat_features)

    model = CatBoostRegressor(iterations=10000,
                              learning_rate=1,
                              depth=5,
                            loss_function='RMSE', verbose=False)
    # Fit model
    model.fit(data_pool)

    with open(f'models/{cur_feat}.pickle', 'wb') as f:
        pickle.dump(model, f)

100%|████████████████████████████████████████████████████████████████████████████████████| 50/50 [10:29<00:00, 12.59s/it]
